In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def dydx(x,y):
    y_derivs = np.zeros(2)
    
    y_derivs[0] = y[1]
    
    y_derivs[1] = -1*y[0]
    
    return y_derivs

In [ ]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    x_ipoh_k2 = xi + 1/5*h
    x_ipoh_k3 = xi + 3/10*h
    x_ipoh_k4 = xi + 3/5*h
    x_ipoh_k5 = xi + h
    x_ipoh_k6 = xi + 7/8*h
    
    x_ipo = xi + h
    
    
    y_temp = np.zeros(nv)
    
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 1/5*k1[:]
    y_derivs = dydx(x_ipoh_k2,y_temp)
    k2[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 3/40*k1[:]+9/40*k2[:]
    y_derivs = dydx(x_ipoh_k3,y_temp)
    k3[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 3/10*k1[:] + -9/10*k2[:] + 6/5*k3[:]
    y_derivs = dydx(x_ipoh_k4,y_temp)
    k4[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + -11/54*k1[:] + 5/2*k2[:] + -70/27*k3[:] + 35/27*k4[:]
    y_derivs = dydx(x_ipoh_k5,y_temp)
    k5[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 1631/55296*k1[:] + 175/512*k2[:] + 575/13824*k3[:] + 44275/110592*k4[:]+ 253/4096*k5[:]
    y_derivs = dydx(x_ipoh_k6,y_temp)
    k6[:] = h*y_derivs[:]
    
   
    y1 = yi[:] + (37/378*k1[:] + 0*k2[:] +250/621*k3[:] + 125/594*k4[:] +0*k5[:]+ 512/1771*k6[:])
    y2 = yi[:] + (2825/27648*k1[:] +0*k2[:]+18575/48384*k3[:]+13525/55296*k4[:]+277/14336*k5[:]+1/4*k6[:])
    
    return y1,y2

In [ ]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    imax = 10000
    
    i = 0
    
    Delta = np.full(nv,2*tol)
    
    h_step = h
    
    while(Delta.max()/tol > 1.0):
            
        Delta = np.fabs(y1-y2)
        
        if(Delta.max()/tol > 1.0):
            
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
        if(i>=imax):
            print("Too many iterations in rk_mv_ad()")
            raise StopIteration("Ending after i = ",i)
        i+=1
    
    h_new = np.fmin(h_step *(Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    return y_2,h_new,h_step

In [ ]:
def rk4_mv(dydx,a,b,y_a,tol):
    xi = a
    yi = y_a.copy()
    
    h = 1.0e-4 * (b-a)
    imax = 10000
    
    i = 0
    nv = len(y_a)
    
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    flag = 1
    
    while(flag):
        yi_new,h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        h = h_new
        
        if(xi+h_step>b):
            
            h = b-xi
            
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            flag = 0
        xi += h_step
        yi[:] = yi_new[:]
        
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        if(i>=imax):
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
        i+=1
        
        s= "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
        
        if(xi==b):
            flag = 0
    return x,y

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2
tolerance = 1.0e-6

x,y = rk4_mv(dydx,a,b,y_0,tolerance)